# Building a GCN from Scratch

<a href="https://colab.research.google.com/github/joerg84/Graph_Powered_ML_Workshop/blob/master/Basic_GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, setting up our environment.

In [ ]:
%%capture
!git clone https://github.com/joerg84/Graph_Powered_ML_Workshop.git
!rsync -av Graph_Powered_ML_Workshop/ ./ --exclude=.git
!pip3 install dgl
!pip3 install numpy
!pip3 install torch
!pip3 install networkx
!pip3 install matplotlib

In [ ]:
%matplotlib inline

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

Let us built a toy GCN from scratch (inspired by https://towardsdatascience.com/how-to-do-deep-learning-on-graphs-with-graph-convolutional-networks-7d2250723780) 

In [ ]:
#Graph expressed by the adjacency matrix
A = np.matrix([
    [0, 1, 0, 0],
    [0, 0, 1, 1], 
    [0, 1, 0, 0],
    [1, 0, 1, 0]],
    dtype=float
)

# Draw the graph 
G = nx.from_numpy_array(A, create_using=nx.DiGraph())
nx.draw(G, with_labels=True)

Next, we need to add some features to the edges. For simpliclicity we will use two floats (+/- the node's id).

In [ ]:
X = np.matrix([
            [i, -i]
            for i in range(A.shape[0])
        ], dtype=float)

A GCN hidden layer can be seen Hⁱ = f(Hⁱ⁻¹, A)) where A is the adjacency matrix of the graph, Hⁱ⁻¹ is the previous layer (and hence H⁰ = X = feature vector). f() is our respective propagation function specifying how features are aggregated. 
The most basic function imaginable would be f(X, A) = AX (not too meaningful, but helpful to understand the concept).

In [ ]:
h_1 = A * X
print(h_1)

So every node feature is now the sum if its direct neighbours. Note that we are working with a directed graphs and the a directed edge from 0 to 1 indicated that 1 is a neighbour of 0 but note vice versa (so the messages flow opposite to the arrows in the above visualization).

This would represent a GCN with just a single layer and means the features in the output are only influenced by the direct neighbours and not even the node value itself, but we fix that next.

In [ ]:
# We can simply extend our adjacency matrix with a self-loop to preserve the current node value
I = np.matrix(np.eye(A.shape[0]))
A_hat = A + I

h1_hat = A_hat * X
print(h1_hat)

Still nodes with more neighbours will accumulate higher values, but we can use the degree (i.e., number of neighbours) to normalize.

In [ ]:
# First compute the degree matrix of A
D = np.array(np.sum(A, axis=0))[0]
D = np.matrix(np.diag(D))
print(D)
print()

# Also compute the degree matrix of A_hat
D_hat = np.array(np.sum(A_hat, axis=0))[0]
D_hat = np.matrix(np.diag(D_hat))
print(D_hat)
print()

In [ ]:
# Next we use that to normalize our A_hat
A_hat_2 = D_hat**-1 * A_hat

h_hat_2 = A_hat_2 * X
print(h_hat_2)
print()

print("Just to recall the degree of each node (including the self loop):")
print(np.diag(D_hat))

Note that compared to h1_hat the values have been divided by the degree of each node.

Our current propagation function f(X, A) = A_hat_2 * X is static and there are no parameters we could learn.
Let us add weights f(Hⁱ, A) = AHⁱWⁱ and hence allow for our network to be trainable.

In [ ]:
W = np.matrix([
             [1, -1],
             [-1, 1]
         ])

h_hat_3 = A_hat_2 * X * W
(print(h_hat_3))

The last missing piece is to add a non-linear activiation function: f(Hⁱ, A) = σ(AHⁱWⁱ)

In [ ]:
def relu(X):
   return np.maximum(0,X)

h_hat_4 = relu(A_hat_2 * X * W)
(print(h_hat_4))

Congratulation, you have built a GCN hidden layer with adjacency matrix, features, weights and a relu activation function from scratch!

# Back to the Karate Club!

![karate](https://github.com/joerg84/Graph_Powered_ML_Workshop/blob/master/img/karate_club.png?raw=1)

In [ ]:
# We use the karate club representation from networkx
zkc = nx.karate_club_graph()
order = sorted(list(zkc.nodes()))

nx.draw(zkc, with_labels=True)

Let us extract the adjacency matrix:

In [ ]:
A = nx.to_numpy_matrix(zkc, nodelist=order)
print(A)

In [ ]:
# As before add self-loops 
num_nodes = A.shape[0]
I = np.matrix(np.eye(num_nodes))
A_hat = A + I

# And create degree matrix
D_hat = np.array(np.sum(A_hat, axis=0))[0]
D_hat = np.matrix(np.diag(D_hat))

print(D_hat)

As we will learn them later, let us initialize the weights randomly:

In [ ]:
W_1 = np.random.normal(
    loc=0, scale=1, size=(num_nodes, 4))
W_2 = np.random.normal(
    loc=0, size=(W_1.shape[1], 2))

print(W_1)

For readability we create a helper function for the hidden layers:

In [ ]:
def gcn_layer(A_hat, D_hat, X, W):
    return relu(D_hat**-1 * A_hat * X * W)

With that we are ready to specify a two layer GCN:

In [ ]:
# As input we use the identity matrix, i.e., each node is represented as a one-hot encoded categorical variable.
H_0 = np.matrix(np.eye(num_nodes)) 

H_1 = gcn_layer(A_hat, D_hat, H_0, W_1)
H_2 = gcn_layer(A_hat, D_hat, H_1, W_2)

output = H_2
print(output)

Let us look at the output (keeping in mind we haven't trained the network yet and the weights are random!)

In [ ]:
for node in range(34):
    print(node)
    print(output[node])
    print()

In the next notebook we will actually train a GCN.